In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#This program uses Resnet50 fine tuning to classify cats and dogs.
#Standard library
import copy
import re
import random
import glob
import os
import zipfile
import shutil

#Related third party
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import cv2
from keras.utils import np_utils
from matplotlib import pyplot as plt
import pandas as pd

from keras.applications.resnet50 import ResNet50
from keras.models import Model
import tensorflow as tf

import keras
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.layers import Dense, Dropout, Flatten, Activation,GlobalAveragePooling2D
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam,SGD,RMSprop

In [ ]:
#unzip
#Skip this cell if it's already unziped.
zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip').extractall()
zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip').extractall()

In [ ]:
#make dir
#Skip this cell if you have already created the directory
os.makedirs("dataset")
os.makedirs("dataset/train")
os.makedirs("dataset/validation")
os.makedirs("dataset/train/cats")
os.makedirs("dataset/train/dogs")
os.makedirs("dataset/validation/cats")
os.makedirs("dataset/validation/dogs")

In [ ]:
#function definition

def square(img):
    h,w,c = img.shape
    long_edge = max(h,w)
    top = 0
    bottom = 0
    left = 0
    right = 0
    if h < long_edge:
        diff_h = long_edge - h
        top = diff_h // 2
        bottom = diff_h - top
    elif w < long_edge:
        diff_w = long_edge - w
        left = diff_w // 2
        right = diff_w - left
    else:
        pass
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return img

In [ ]:
#pre processing
#If you have already transformed and moved the image, skip this step.
dogs = glob.glob(os.path.join("train","dog.*.jpg"))
cats = glob.glob(os.path.join("train","cat.*.jpg"))    

for file_dog in dogs:
    input = cv2.imread(file_dog)
    input = square(input)
    input = cv2.resize(input,dsize=(224,224))
    cv2.imwrite(file_dog,input)

for file_cat in cats:
    input = cv2.imread(file_cat)
    input = square(input)
    input = cv2.resize(input,dsize=(224,224))
    cv2.imwrite(file_cat,input)

In [ ]:
#parameta definition
img_width,img_height = 224,224
train_size = 20000
validation_size = 5000
batch_size = 32

epochs = 80
learning_rate = 1e-5
test_num = 12500

detect_class = ['dogs','cats']
class_size = len(detect_class)

train_data_dir = './dataset/train'
validation_data_dir = './dataset/validation'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#normalization and image padding
train_data = ImageDataGenerator(rescale = 1.0 / 255, zoom_range = 0.5, horizontal_flip = True,vertical_flip=True,rotation_range=45)
validation_data = ImageDataGenerator(rescale = 1.0 / 255)

train_generator = train_data.flow_from_directory(train_data_dir,target_size=(img_width,img_height),color_mode='rgb',classes = detect_class,class_mode='categorical',batch_size=batch_size)
validation_generator = validation_data.flow_from_directory(validation_data_dir,target_size=(img_width,img_height),color_mode='rgb',classes=detect_class,class_mode = 'categorical',batch_size=batch_size)

#layer parameta definition
input_tensor = Input(shape=(img_width,img_height,3))
base_model = ResNet50(include_top = False,weights='imagenet',input_tensor=input_tensor)
base_model.trainable = True

#model definition
top_model = Sequential()

top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(class_size,activation='softmax'))

model = Model(inputs=base_model.input,outputs=top_model(base_model.output))

#compiler definition
model.compile(loss = 'categorical_crossentropy',optimizer=RMSprop(lr=learning_rate),metrics=['accuracy'])

In [ ]:
#train
history = model.fit_generator(train_generator,steps_per_epoch=train_size // batch_size,epochs=epochs,validation_data=validation_generator,validation_steps=validation_size//batch_size)

In [ ]:
#Visualize accuracy and save parameta

history.history.keys()
fig = plt.figure()
plt.plot(range(1, epochs+1), history.history['accuracy'], label="training")
plt.plot(range(1, epochs+1), history.history['val_accuracy'], label="validation")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

fig.savefig("history.png")
model.save_weights("weights.h5")

In [ ]:
#test and plot csv for submit
from keras.preprocessing import image
import csv

with open("output.csv","a") as file:
    print("id,label",file = file)
    for test_image_num in range(1,test_num + 1):
        img = cv2.imread("test/"+str(test_image_num)+".jpg")
        img = square(img)
        img = cv2.resize(img,dsize=(224,224))
        trans_img = image.img_to_array(img)
        trans_img = np.expand_dims(trans_img, axis=0)
        trans_img = trans_img / 255
        pred = model.predict(trans_img)[0]
        print(pred)
        print(str(test_image_num) + "," + str(pred[0]),file=file)